In [1]:
import pandas as pd

In [2]:
df = pd.read_json("../data/text-davinci-003-single-response.json") \
       .sort_values(["question", "temperature", "frequency_penalty"]) \
       .reset_index(drop=True)
df.head()

,question,response,model,max_tokens,stop,prompt,temperature,frequency_penalty
0,How do birthday cakes make you feel?,Birthday cakes make me feel both happy and sad...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.01
1,How do birthday cakes make you feel?,Birthday cakes make me feel nostalgic. I alway...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.01
2,How do birthday cakes make you feel?,Birthdays cakes make me feel excited and happy...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.01
3,How do birthday cakes make you feel?,Birthday cakes make me feel nostalgic and happ...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.10
4,How do birthday cakes make you feel?,Birthday cakes make me feel really happy and e...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.10


In [3]:
df.to_excel("../data/text-davinci-003-single-response.xlsx", index=False)

In [5]:
#import random 
import random
def random_generate(lower = -1000, upper = 1000, num = 30):
    testx = []
    testy = []
    for i in range(num):
        testx.append(random.randint(lower, upper))
        testy.append(random.randint(lower, upper))
    return testx, testy


In [7]:
a,b = random_generate()

[-240,
 216,
 269,
 -919,
 -665,
 -52,
 328,
 -740,
 -825,
 127,
 -734,
 885,
 -574,
 -103,
 -529,
 886,
 -748,
 -13,
 940,
 292,
 -638,
 -560,
 -83,
 -45,
 926,
 700,
 -938,
 -148,
 164,
 -584]

In [39]:
def fs(x):
    hash = set([1,1])
    prev = 1
    curr = 1
    for i in range(1000):
        tmp = prev + curr 
        prev = curr
        curr = tmp
        hash.add(curr)
    print(len(str(curr)))
    ret = []

    for i in x:
        condition = False
        if i == 2:
            ret.append(True)
            continue
        else:
            condition = False
            for n in hash:
                if i-n in hash:
                    ret.append(True)
                    condition = True
                    break
            if condition == False:
                ret.append(False)
    return ret



In [40]:
fs([2,2,5])

210


[True, True, True]

In [43]:
x,y = random_generate(lower = 0, upper = 10**18, num = 5000)

In [44]:
import timeit

start = timeit.default_timer()
fs(x)
stop = timeit.default_timer()
print('Time: ', stop - start)

210
Time:  0.4764957000006689
